# 姿勢制御、着陸を学習する

Stable BaselinesとRL Baselines Zooを用いて、強化学習による姿勢制御、着陸ゲーム(Lunar Lander)の学習を試します。

まずStable Baselinesで学習の詳細の流れを確認したあと、CartPoleでも最後に紹介した、RL Baselines Zooのお手軽1行コマンドで学習、結果の可視化を行います。

このノートブックは以下の内容を含みます。

- 環境準備
- Gym環境とエージェントを作成
- エージェントの学習と評価
- 1行のコマンドで学習
- リプレイ動画の生成

このノートブックの、未学習エージェント、および学習済みエージェントの振る舞いの観察するまではX分で実行できます。
実際にエージェントを学習させ、振る舞いを観察するには、追加でX分程度を要します。


なお、GIFアニメーションによる学習前後のプレイの可視化は、ikeyasu氏の[ChainerRL を Colaboratory で動かす - Qiita](https://qiita.com/ikeyasu/items/ec3c88ce13a2d5e41f26) を参考として作成しました。

# LunarLander-v2

LunarLander-v2環境は、離散値で制御します。

## A. 環境を準備する

Stable Baselinesと依存ライブラリをインストールします。

### 1. 必要なライブラリのインストール

インストールに5分程度を要します。

In [0]:
!apt-get -y install swig xvfb python-opengl
!pip install box2d box2d-kengz pybullet pyyaml pytablewriter pyvirtualdisplay

### 2. 必要なPythonライブラリのインポート


In [0]:
import time, os, gym
import numpy as np

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C, PPO2

from pyvirtualdisplay import Display

import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML

## B. Gym環境を準備する



In [0]:
env = gym.make('LunarLander-v2')
env = DummyVecEnv([lambda: env])  # The algorithms require a vectorized environment to run

## C. その他の処理を準備する

### 1. 評価方法の準備

エージェントを評価するためのヘルパー関数を作成します。

In [0]:
def evaluate(model, num_steps=1000):
  """
  Evaluate a RL agent
  :param model: (BaseRLModel object) the RL Agent
  :param num_steps: (int) number of timesteps to evaluate it
  :return: (float) Mean reward for the last 100 episodes
  """
  episode_rewards = [0.0]
  obs = env.reset()
  for i in range(num_steps):
      # _states are only useful when using LSTM policies
      action, _states = model.predict(obs)
      # here, action, rewards and dones are arrays
      # because we are using vectorized env
      obs, rewards, dones, info = env.step(action)
      
      # Stats
      episode_rewards[-1] += rewards[0]
      if dones[0]:
          obs = env.reset()
          episode_rewards.append(0.0)
  # Compute mean reward for the last 100 episodes
  mean_100ep_reward = round(np.mean(episode_rewards[-100:]), 1)
  print("Mean reward:", mean_100ep_reward, "Num episodes:", len(episode_rewards))
  
  return mean_100ep_reward

### 2. プレイ動画の再生用関数

次に、仮想ディプレイを利用し、Colaboratory上でエージェントの振る舞いをアニメーションで見られるようにする関数を定義します。

In [0]:
def playback(model, env, maxsteps):
  # Start virtual display
  display = Display(visible=0, size=(1024, 768))
  display.start()

  os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

  frames = []
  for i in range(3):
      obs = env.reset()
      done = False
      R = 0
      t = 0
      while not done and t < maxsteps:
          frames.append(env.render(mode = 'rgb_array'))
          action, _states = model.predict(obs)        
          obs, rewards, dones, info = env.step(action)
          R += rewards
          t += 1
      print('test episode:', i, 'R:', R)
  #    model.stop_episode()
  #env.render()

  return frames

## D. モデルの作成

### 1. エージェントの準備

A2Cというアルゴリズムで、エージェントを作成します。

In [0]:
model = A2C(MlpPolicy, env, ent_coef=0.1, verbose=0, tensorboard_log="./lunar_tensorboard/")

### 3. 学習前のエージェントの評価

実行すると平均の報酬(Mean reward)と、エピソード数(Num episodes)が得られます。学習後の実行と比べてみましょう。

In [0]:
# Random Agent, before training
mean_reward_before_train = evaluate(model, num_steps=10000)

### 4. 未学習状態での振る舞いを見る

さて、この未学習状態でのプレイぶりを再生してみましょう。

In [0]:
frames = playback(model, env, 100)

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
HTML(ani.to_jshtml())

## E. エージェントを学習させる

### 1. TensorBoardのセットアップ

学習の経過をモニタするため、Tensorboardをセットアップします。セルの出力に、インラインで表示することができます。

In [0]:
%load_ext tensorboard.notebook
%tensorboard --logdir lunar_tensorboard

### 2. エージェントを学習させる

試しに10,000ステップ学習を進めてみます。所要時間は1分程度です。


In [0]:
model.learn(total_timesteps=10000)

### 3. 学習済みエージェントの読み込み

In [0]:
#model = A2C.load("a2c_lunar")
model = A2C.load("rl-baselines-zoo/trained_agents/a2c/LunarLander-v2.pkl")
env = gym.make('LunarLander-v2')
# vectorized environments allow to easily multiprocess training
# we demonstrate its usefulness in the next examples
env = DummyVecEnv([lambda: env]) 
model.set_env(env)

## F. 学習済みエージェントの評価

### 1. 学習済みエージェントの評価

学習後の平均の報酬(Mean reward)と、エピソード数(Num episodes)が得られます。学習前と比べてみましょう。

In [0]:
# Evaluate the trained agent
mean_reward = evaluate(model, num_steps=10000)

### 2. 学習済みエージェントの振る舞いを見る

実際の振る舞いを再生してみましょう。学習前より着陸がうまくできているでしょうか。学習前よりは多少改善しているようです。

In [0]:
frames = playback(model, env, 300)

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
HTML(ani.to_jshtml())

### 3. RL Baselines Zooのインストール

今回も、ここからは、

- 環境を準備する
- エージェントを作成する
- 環境上で、エージェントに探索させる

というステップを1行のコマンドの裏側にまとめてくれるRL Baselines Zooを使ってみましょう。

In [0]:
%cd /content
!git clone https://github.com/araffin/rl-baselines-zoo.git

### 4. 1行のコマンドで学習する

実行すると、 `logs/a2c/LunarLander-v2_1/` に  `LunarLander-v2.pkl` として、学習済みエージェントのモデルが保存されます。(一般化すると、 `logs/{algorithm}/{env_name}_n` に、 `{env_name}.pkl` として保存されます。)

学習の条件は予め最適化されています。今回の場合、16環境を並列させ、秒間1700フレームあまりの高速で学習を進めます。



In [0]:
%cd /content/rl-baselines-zoo
!python train.py --algo a2c --env LunarLander-v2 --tensorboard-log lunar_tensorboard/

### 5. 学習済みエージェントを読み込み、プレイバック

では、学習結果を見てみましょう。学習を複数回実行した場合は、`logs/a2c/LunarLander-v2_1/LunarLander-v2.pkl` の、nの数を変えてください。

In [0]:
%cd /content/rl-baselines-zoo
env_id = "LunarLander-v2"
model_path = "logs/a2c/LunarLander-v2_1/LunarLander-v2.pkl"

env = gym.make(env_id)
env = DummyVecEnv([lambda: env]) 
model = A2C.load(model_path)
model.set_env(env)

次に再生をします。うまく着陸できるようになったでしょうか。

In [0]:
frames = playback(model, env, 500)

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
HTML(ani.to_jshtml())

# LunarLanderContinuous-v2

LunarLanderContinuous-v2環境は、Continuousと名前にある通り、連続値で制御します。

## A. 環境を準備する

LunarLander-v2と共通のため、割愛します。

## B. Gym環境を準備する

今回はRL Baselines Zooでまとめて学習するため、割愛します。


## C. その他の処理を準備する

LunarLander-v2と共通のため、割愛します。

## D. モデルの作成

今回はRL Baselines Zooでまとめて学習するため、割愛します。

## E. エージェントを学習させる

### TensorBoardで学習をモニタする

再度モニタ用のTensorBoardをセットアップしましょう。

In [0]:
%load_ext tensorboard.notebook
%tensorboard --logdir lunar_tensorboard/LunarLanderContinuous-v2

### 1行のコマンドで学習する

RL Baselines Zooをインストール済みであるとして進めます。

今回は連続値による制御であるため、PPO2をエージェントのモデルに用います。

実行すると、 `logs/ppo2/LunarLanderContinuous-v2_1/` に  `LunarLanderContinuous-v2.pkl` として、学習済みエージェントのモデルが保存されます。


In [0]:
%cd /content/rl-baselines-zoo
!python train.py --algo ppo2 --env LunarLanderContinuous-v2 --tensorboard-log lunar_tensorboard/

### 3. 学習済みエージェントを読み込み、プレイバック

では、学習結果を見てみましょう。学習を複数回実行した場合は、`logs/ppo2/LunarLanderContinuous-v2_n/LunarLanderContinuous-v2.pkl` の、nの数を変えてください。

In [0]:
%cd /content/rl-baselines-zoo
env_id = "LunarLanderContinuous-v2"
model_path = "logs/ppo2/LunarLanderContinuous-v2_1/LunarLanderContinuous-v2.pkl"

model = PPO2.load(model_path)
env = gym.make(env_id)
env = DummyVecEnv([lambda: env]) 
model.set_env(env)

次に再生をします。

In [0]:
frames = playback(model, env, 500)

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
HTML(ani.to_jshtml())

## まとめと発展

このノートブックでは、LunarLanderを、離散値の制御、連続値の制御2つの異なるパターンで、モデルの種類を変えて習熟させることができました。
